# The four steps of causal inference

## I. Model a causal problem
- Create a causal DAG for your system of interest.

## II. Identify a target estimand under the model
- Identify the causal estimand under the assumptions of the causal DAG.

## III. Estimate the causal effect based on the identified estimand
- Estimate the estimand using any kind of Stats/ML model e.g. linear regression, random forest etc.

## IV. Refute the obtain estimate
- Peform refutations on the estimate to test its robustness 

## Imports

In [1]:
import numpy as np
import pandas as pd
import patsy as ps
import statsmodels
import dowhy
from dowhy import CausalModel
import econml
from IPython.display import Image, display
from pathlib import Path
import os
import sys

# Avoid printing dataconversion warnings from sklearn
# Config dict to set the logging level
import logging.config
import warnings
warnings.filterwarnings('ignore')
DEFAULT_LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'loggers': {
        '': {
            'level': 'WARN',
        },
    }
}
logging.config.dictConfig(DEFAULT_LOGGING)

# Import custom dowhy helper functions module
cwd = Path().resolve()
PARENT_DIR = os.path.dirname(cwd)
SCRIPT_DIR = os.path.join(PARENT_DIR, 'helpers')
sys.path.append(SCRIPT_DIR)
import dowhy_helpers as dwh

In [2]:
# I/O Stuff
DATA_FILENAME = "csdh_clean.csv"
DATA_FILEPATH = "/Users/callum/Uni/GitHubRepos/surviving-the-icu/datasets/drain_data/" + DATA_FILENAME
csdh = pd.read_csv(DATA_FILEPATH)

---
## 0. Naïve Estimation (no causal inference)

In [3]:
naive_est = dwh.naive_estimate(df=csdh, treatment='drain', outcome='recurrence', treatment_type='int')
print(f"Without adjusting for any confounding, the naive causal estimate is computed as {naive_est}")

Without adjusting for any confounding, the naive causal estimate is computed as -0.09356128931064231


---
## I. Model a causal problem
* Create a causal model from the data and given graph.

In [4]:
mp_model = CausalModel(data=csdh, 
                       treatment='drain', 
                       outcome='recurrence', 
                       graph='../causal_graphs/mp_dag.dot'.replace("\n", " "))

In [5]:
data_model = CausalModel(data=csdh, 
                         treatment='drain', 
                         outcome='recurrence', 
                         graph='../causal_graphs/data_dag.dot'.replace("\n", " "))

In [6]:
small_data_model = CausalModel(data=csdh,
                               treatment='drain', 
                               outcome='recurrence', 
                               graph='../causal_graphs/small_data_dag.dot'.replace("\n", " "))

---
## II. Identify a target estimand under the model

In [7]:
mp_estimand = mp_model.identify_effect(proceed_when_unidentifiable=True)
data_estimand = data_model.identify_effect(proceed_when_unidentifiable=True)
small_data_estimand = small_data_model.identify_effect(proceed_when_unidentifiable=True)

---
## III. Linear Regression Estimator

In [8]:
# III. Estimate the target estimand using a statistical method.
mp_lin_est = dwh.linear_regression_estimator(mp_model, mp_estimand, ci=True, test_significance=True)
data_lin_est = dwh.linear_regression_estimator(data_model, data_estimand, ci=True, test_significance=True)
sdata_lin_est = dwh.linear_regression_estimator(small_data_model, small_data_estimand, ci=True, test_significance=True)

In [9]:
print(mp_lin_est)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                                                                          
────────(Expectation(recurrence|antiplatelet,thickness_sum,age,adm_mrs,gcs_pre
d[drain]                                                                      

    
op))
    
Estimand assumption 1, Unconfoundedness: If U→{drain} and U→recurrence then P(recurrence|drain,antiplatelet,thickness_sum,age,adm_mrs,gcs_preop,U) = P(recurrence|drain,antiplatelet,thickness_sum,age,adm_mrs,gcs_preop)

## Realized estimand
b: recurrence~drain+antiplatelet+thickness_sum+age+adm_mrs+gcs_preop+drain*antiplatelet
Target units: ate

## Estimate
Mean value: -0.10562889672174253
p-value: [0.01406595]
95.0% confidence interval: [[-0.14587421 -0.01642682]]
### Conditional Estimates
__categorical__antiplatelet
(-0.001, 1.0]   -0.105629
dtype: float64


In [10]:
print(data_lin_est)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                                                                          
────────(Expectation(recurrence|midlineshift,copd,steroid,density,inr,epilepsy
d[drain]                                                                      

                                                                              
,platelet,bilateral,ihd,age,arrhythmia,antiplatelet,metalvalve,stroke,thicknes
                                                                              

                           
s_sum,membranes,gcs_preop))
                           
Estimand assumption 1, Unconfoundedness: If U→{drain} and U→recurrence then P(recurrence|drain,midlineshift,copd,steroid,density,inr,epilepsy,platelet,bilateral,ihd,age,arrhythmia,antiplatelet,metalvalve,stroke,thickness_sum,membranes,gcs_preop,U) = P(recurrence|drain,midlineshift,copd,steroid,d

In [11]:
print(sdata_lin_est)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                                                                          
────────(Expectation(recurrence|antiplatelet,inr,metalvalve,platelet,stroke,ih
d[drain]                                                                      

                     
d,thickness_sum,age))
                     
Estimand assumption 1, Unconfoundedness: If U→{drain} and U→recurrence then P(recurrence|drain,antiplatelet,inr,metalvalve,platelet,stroke,ihd,thickness_sum,age,U) = P(recurrence|drain,antiplatelet,inr,metalvalve,platelet,stroke,ihd,thickness_sum,age)

## Realized estimand
b: recurrence~drain+antiplatelet+inr+metalvalve+platelet+stroke+ihd+thickness_sum+age+drain*metalvalve+drain*ihd
Target units: ate

## Estimate
Mean value: -0.10849711522588248
p-value: [0.02185382]
95.0% confidence interval: [[-0.1382241  -0.01085408]]
### Conditional Estimate

In [12]:
# mp model
dwh.print_estimate_comparison(naive_est, mp_lin_est, 'Linear regression')

-------------- Causal Estimates -------------- 
Naive causal estimate is -0.09356128931064231
Linear regression causal estimate is -0.10562889672174253
Percentage change from naive_est: 12.898%
----------------------------------------------


In [13]:
# small data model
dwh.print_estimate_comparison(naive_est, sdata_lin_est, 'Linear regression')

-------------- Causal Estimates -------------- 
Naive causal estimate is -0.09356128931064231
Linear regression causal estimate is -0.10849711522588248
Percentage change from naive_est: 15.964%
----------------------------------------------


In [14]:
# data model
dwh.print_estimate_comparison(naive_est, data_lin_est, 'Linear regression')

-------------- Causal Estimates -------------- 
Naive causal estimate is -0.09356128931064231
Linear regression causal estimate is -0.11644725221958863
Percentage change from naive_est: 24.461%
----------------------------------------------


---
## IV. Refute the obtained estimate

1. **Add Random Common Cause:** Does the estimation method change its estimate after we add an independent random variable as a common cause to the dataset? (Hint: It should not)

In [15]:
# Robust if: estimate stays the same
lin_ran_refuter = mp_model.refute_estimate(mp_estimand, mp_lin_est, 
                                           method_name="random_common_cause", num_simulations=100)

In [33]:
lin_ran_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, sdata_lin_est, 
                                                   method_name="random_common_cause", num_simulations=100)

In [17]:
lin_ran_refuter_data = data_model.refute_estimate(data_estimand, data_lin_est, 
                                                  method_name="random_common_cause", num_simulations=100)

In [34]:
print(lin_ran_refuter)
print(lin_ran_refuter_sdata)
print(lin_ran_refuter_data)

Refute: Add a Random Common Cause
Estimated effect:-0.10562889672174253
New effect:-0.10576178808664236

Refute: Add a Random Common Cause
Estimated effect:-0.10849711522588248
New effect:-0.106878154928222

Refute: Add a Random Common Cause
Estimated effect:-0.11644725221958863
New effect:-0.11580714074147552



2. **Placebo Treatment:** What happens to the estimated causal effect when we replace the true treatment variable with an independent random variable? (Hint: the effect should go to zero)



* Note that the placebo type is 'permute' meaning the rows of the treatment variable have been randomly permuted giving the effect of a placebo treatment.


In [19]:
# Robust if: estimate goes to 0
lin_placebo_refuter = mp_model.refute_estimate(mp_estimand, mp_lin_est, 
                                               method_name="placebo_treatment_refuter",
                                               num_simulations=100,
                                               placebo_type='permute')

lin_placebo_refuter_data = data_model.refute_estimate(data_estimand, data_lin_est, 
                                                      method_name="placebo_treatment_refuter",
                                                      num_simulations=100, 
                                                      placebo_type='permute')

lin_placebo_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, sdata_lin_est, 
                                                      method_name="placebo_treatment_refuter",
                                                      num_simulations=100, 
                                                      placebo_type='permute')

In [20]:
print(lin_placebo_refuter)
print(lin_placebo_refuter_data)
print(lin_placebo_refuter_sdata)

Refute: Use a Placebo Treatment
Estimated effect:-0.10562889672174253
New effect:-0.002291339975030228
p value:0.49

Refute: Use a Placebo Treatment
Estimated effect:-0.11644725221958863
New effect:-0.002208885717177781
p value:0.5

Refute: Use a Placebo Treatment
Estimated effect:-0.10849711522588248
New effect:0.0006689357270149696
p value:0.49



3. **Dummy Outcome:** What happens to the estimated causal effect when we replace the true outcome variable with an independent random variable? (Hint: The effect should go to zero)



* The result shows that when using a dummy outcome, the **treatment does not lead to the outcome**. The estimated effect is hence a value that tends to zero, which matches our expectation. This shows that if we replace the outcome by randomly generated data, the **estimator correctly predicts that the influence if treatment is zero**.


In [21]:
# Robust if: estimate goes to 0
lin_dummy_refuter = mp_model.refute_estimate(mp_estimand, mp_lin_est, 
                                             method_name="dummy_outcome_refuter", num_simulations=100)
lin_dummy_refuter_data = data_model.refute_estimate(data_estimand, data_lin_est,
                                                    method_name="dummy_outcome_refuter", num_simulations=100)

lin_dummy_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, sdata_lin_est,
                                                    method_name="dummy_outcome_refuter", num_simulations=100)


In [22]:
print(lin_dummy_refuter[0])
print(lin_dummy_refuter_data[0])
print(lin_dummy_refuter_sdata[0])


Refute: Use a Dummy Outcome
Estimated effect:0
New effect:-0.00921745019832052
p value:0.49

Refute: Use a Dummy Outcome
Estimated effect:0
New effect:-0.025462638453289995
p value:0.32999999999999996

Refute: Use a Dummy Outcome
Estimated effect:0
New effect:0.009561667503199412
p value:0.44



4. **Data Subsets Validation:** Does the estimated effect change significantly when we replace the given dataset with a randomly selected subset? (Hint: It should not)

In [23]:
# Robust if: estimate stays the same
lin_subset_refuter = mp_model.refute_estimate(mp_estimand, mp_lin_est, 
                                              method_name="data_subset_refuter",
                                              num_simulations=100,
                                              subset_fraction=0.75)
lin_subset_refuter_data = data_model.refute_estimate(data_estimand, data_lin_est, 
                                                     method_name="data_subset_refuter",
                                                     num_simulations=100,
                                                     subset_fraction=0.75)
lin_subset_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, sdata_lin_est, 
                                                     method_name="data_subset_refuter",
                                                     num_simulations=100,
                                                     subset_fraction=0.75)

In [24]:
print(lin_subset_refuter)
print(lin_subset_refuter_data)
print(lin_subset_refuter_sdata)

Refute: Use a subset of data
Estimated effect:-0.10562889672174253
New effect:-0.10794866476919793
p value:0.43999999999999995

Refute: Use a subset of data
Estimated effect:-0.11644725221958863
New effect:-0.11363650527126827
p value:0.48

Refute: Use a subset of data
Estimated effect:-0.10849711522588248
New effect:-0.10894847922588217
p value:0.45999999999999996



5. **Bootstrap Validation:** Does the estimated effect change significantly when we replace the given dataset with bootstrapped samples from the same dataset? (Hint: It should not)

In [25]:
# Robust if: estimate stays the same
lin_bootstrap_refuter = mp_model.refute_estimate(mp_estimand, mp_lin_est, 
                                                 method_name="bootstrap_refuter",
                                                 num_simulations=100)

lin_bootstrap_refuter_data = data_model.refute_estimate(data_estimand, data_lin_est, 
                                                        method_name="bootstrap_refuter", 
                                                        num_simulations=100)

lin_bootstrap_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, sdata_lin_est, 
                                                               method_name="bootstrap_refuter", 
                                                               num_simulations=100)

In [26]:
print(lin_bootstrap_refuter)
print(lin_bootstrap_refuter_data)
print(lin_bootstrap_refuter_sdata)

Refute: Bootstrap Sample Dataset
Estimated effect:-0.10562889672174253
New effect:-0.10338488270601362
p value:0.44

Refute: Bootstrap Sample Dataset
Estimated effect:-0.11644725221958863
New effect:-0.11871180345215784
p value:0.43999999999999995

Refute: Bootstrap Sample Dataset
Estimated effect:-0.10849711522588248
New effect:-0.10657350850129578
p value:0.49



---
## III. Generalized linear model (GLM) estimator


In [27]:
mp_glm_est = dwh.bin_glm_estimator(mp_model, mp_estimand, ci=True, test_significance=True)

In [28]:
data_glm_est = dwh.bin_glm_estimator(data_model, data_estimand, ci=True, test_significance=True)

In [29]:
sdata_glm_est = dwh.bin_glm_estimator(small_data_model, small_data_estimand, ci=True, test_significance=True)

In [30]:
print(mp_glm_est)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                                                                          
────────(Expectation(recurrence|antiplatelet,thickness_sum,age,adm_mrs,gcs_pre
d[drain]                                                                      

    
op))
    
Estimand assumption 1, Unconfoundedness: If U→{drain} and U→recurrence then P(recurrence|drain,antiplatelet,thickness_sum,age,adm_mrs,gcs_preop,U) = P(recurrence|drain,antiplatelet,thickness_sum,age,adm_mrs,gcs_preop)

## Realized estimand
b: recurrence~Sigmoid(drain+antiplatelet+thickness_sum+age+adm_mrs+gcs_preop+drain*antiplatelet)
Target units: ate

## Estimate
Mean value: -0.11124053164452738
p-value: [0, 0.001]
95.0% confidence interval: (-0.16952364283819177, -0.04551411754570943)
### Conditional Estimates
__categorical__antiplatelet
(-0.001, 1.0]   -0.111241
dtype: float64


In [31]:
print(data_glm_est)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                                                                          
────────(Expectation(recurrence|midlineshift,copd,steroid,density,inr,epilepsy
d[drain]                                                                      

                                                                              
,platelet,bilateral,ihd,age,arrhythmia,antiplatelet,metalvalve,stroke,thicknes
                                                                              

                           
s_sum,membranes,gcs_preop))
                           
Estimand assumption 1, Unconfoundedness: If U→{drain} and U→recurrence then P(recurrence|drain,midlineshift,copd,steroid,density,inr,epilepsy,platelet,bilateral,ihd,age,arrhythmia,antiplatelet,metalvalve,stroke,thickness_sum,membranes,gcs_preop,U) = P(recurrence|drain,midlineshift,copd,steroid,d

In [32]:
print(sdata_glm_est)

*** Causal Estimate ***

## Identified estimand
Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
   d                                                                          
────────(Expectation(recurrence|antiplatelet,inr,metalvalve,platelet,stroke,ih
d[drain]                                                                      

                     
d,thickness_sum,age))
                     
Estimand assumption 1, Unconfoundedness: If U→{drain} and U→recurrence then P(recurrence|drain,antiplatelet,inr,metalvalve,platelet,stroke,ihd,thickness_sum,age,U) = P(recurrence|drain,antiplatelet,inr,metalvalve,platelet,stroke,ihd,thickness_sum,age)

## Realized estimand
b: recurrence~Sigmoid(drain+antiplatelet+inr+metalvalve+platelet+stroke+ihd+thickness_sum+age+drain*metalvalve+drain*ihd)
Target units: ate

## Estimate
Mean value: -0.11013202824956747
p-value: [0, 0.001]
95.0% confidence interval: (-0.16274668793359107, -0.04835436632903764)


---
## IV. Refute the obtained estimate

1. **Add Random Common Cause:** Does the estimation method change its estimate after we add an independent random variable as a common cause to the dataset? (Hint: It should not)

In [43]:
# Robust if: estimate stays the same
glm_ran_refuter = mp_model.refute_estimate(mp_estimand, mp_glm_est, method_name="random_common_cause", num_simulations=100)

In [45]:
glm_ran_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, data_glm_est, method_name="random_common_cause", num_simulations=100)

In [47]:
glm_ran_refuter_data = data_model.refute_estimate(data_estimand, data_glm_est, method_name="random_common_cause", num_simulations=100)

In [46]:
print(glm_ran_refuter)
print(glm_ran_refuter_sdata)
print(glm_ran_refuter_data)

Refute: Add a Random Common Cause
Estimated effect:-0.11124053164452738
New effect:-0.11000215245384974

Refute: Add a Random Common Cause
Estimated effect:-0.12644876282215634
New effect:-0.11086691424267466

Refute: Add a Random Common Cause
Estimated effect:-0.12644876282215634
New effect:-0.12591332723906165



2. **Placebo Treatment:** What happens to the estimated causal effect when we replace the true treatment variable with an independent random variable? (Hint: the effect should go to zero)



* Note that the placebo type is 'permute' meaning the rows of the treatment variable have been randomly permuted giving the effect of a placebo treatment.


In [59]:
# Robust if: estimate goes to 0
glm_placebo_refuter = mp_model.refute_estimate(mp_estimand, 
                                               mp_glm_est,
                                               method_name="placebo_treatment_refuter",
                                               num_simulations=100,
                                               placebo_type='permute')


In [60]:
glm_placebo_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, 
                                                             sdata_glm_est, 
                                                             method_name="placebo_treatment_refuter",
                                                             num_simulations=100,
                                                             placebo_type='permute')

In [61]:
glm_placebo_refuter_data = data_model.refute_estimate(data_estimand, 
                                                      data_glm_est,
                                                      method_name="placebo_treatment_refuter",
                                                      num_simulations=100,
                                                      placebo_type='permute')

In [62]:
print(glm_placebo_refuter)
print(glm_placebo_refuter_sdata)
print(glm_placebo_refuter_data)

Refute: Use a Placebo Treatment
Estimated effect:-0.11124053164452738
New effect:0.0017622580657989811
p value:0.47

Refute: Use a Placebo Treatment
Estimated effect:-0.11013202824956747
New effect:0.0002449093988948427
p value:0.5

Refute: Use a Placebo Treatment
Estimated effect:-0.12644876282215634
New effect:-0.0014730608287027661
p value:0.47



3. **Dummy Outcome:** What happens to the estimated causal effect when we replace the true outcome variable with an independent random variable? (Hint: The effect should go to zero)



* The result shows that when using a dummy outcome, the **treatment does not lead to the outcome**. The estimated effect is hence a value that tends to zero, which matches our expectation. This shows that if we replace the outcome by randomly generated data, the **estimator correctly predicts that the influence if treatment is zero**.

In [67]:
# Robust if: estimate goes to 0
glm_dummy_refuter = mp_model.refute_estimate(mp_estimand, mp_glm_est,
                                             method_name="dummy_outcome_refuter", 
                                             num_simulations=100)

In [68]:
glm_dummy_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, 
                                                           sdata_glm_est, 
                                                           method_name="dummy_outcome_refuter",
                                                           num_simulations=100)

In [69]:
glm_dummy_refuter_data = data_model.refute_estimate(data_estimand, 
                                                    data_glm_est, 
                                                    method_name="dummy_outcome_refuter",
                                                    num_simulations=100)

In [79]:
print(glm_dummy_refuter[0])
print(glm_dummy_refuter_sdata[0])
print(glm_dummy_refuter_data[0])

Refute: Use a Dummy Outcome
Estimated effect:0
New effect:-0.03176279526577422
p value:0.76

Refute: Use a Dummy Outcome
Estimated effect:0
New effect:-0.025234899328859056
p value:0.43999999999999995

Refute: Use a Dummy Outcome
Estimated effect:0
New effect:-0.022697986577181205
p value:0.4



4. **Data Subsets Validation:** Does the estimated effect change significantly when we replace the given dataset with a randomly selected subset? (Hint: It should not)

In [71]:
# Robust if: estimate stays the same
glm_subset_refuter = mp_model.refute_estimate(mp_estimand, mp_glm_est, method_name="data_subset_refuter", 
                                              subset_fraction=0.5, num_simulations=100)

In [72]:
glm_subset_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, sdata_glm_est, 
                                                            method_name="data_subset_refuter", 
                                                            subset_fraction=0.5, num_simulations=100)

In [73]:
glm_subset_refuter_data = data_model.refute_estimate(data_estimand, data_glm_est, 
                                                     method_name="data_subset_refuter", 
                                                     subset_fraction=0.5, num_simulations=100)

In [74]:
print(glm_subset_refuter)
print(glm_subset_refuter_sdata)
print(glm_subset_refuter_data)

Refute: Use a subset of data
Estimated effect:-0.11124053164452738
New effect:-0.10203874150722521
p value:0.42

Refute: Use a subset of data
Estimated effect:-0.11013202824956747
New effect:-0.09882132792286683
p value:0.36

Refute: Use a subset of data
Estimated effect:-0.12644876282215634
New effect:-0.12538736013487936
p value:0.48



5. **Bootstrap Validation:** Does the estimated effect change significantly when we replace the given dataset with bootstrapped samples from the same dataset? (Hint: It should not)

In [75]:
# Robust if: estimate stays the same
glm_bootstrap_refuter = mp_model.refute_estimate(mp_estimand, mp_glm_est, method_name="bootstrap_refuter", 
                                                 num_simulations=100)

In [76]:
glm_bootstrap_refuter_sdata = small_data_model.refute_estimate(small_data_estimand, sdata_glm_est, 
                                                               method_name="bootstrap_refuter", 
                                                               num_simulations=100)

In [77]:
glm_bootstrap_refuter_data = data_model.refute_estimate(data_estimand, data_glm_est, 
                                                        method_name="bootstrap_refuter", 
                                                        num_simulations=100)

In [78]:
print(glm_bootstrap_refuter)
print(glm_bootstrap_refuter_sdata)
print(glm_bootstrap_refuter_data)

Refute: Bootstrap Sample Dataset
Estimated effect:-0.11124053164452738
New effect:-0.10909803479930678
p value:0.45

Refute: Bootstrap Sample Dataset
Estimated effect:-0.11013202824956747
New effect:-0.10819529981946285
p value:0.45

Refute: Bootstrap Sample Dataset
Estimated effect:-0.12644876282215634
New effect:-0.12133007442924862
p value:0.48

